Plataforma de Streaming

Transformaciones:	
	Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.
	DESANIDAR DATOS ANIDADOS
	Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
	REVENUE, BUDGET; NULOS RELLENAR CON 0
	Los valores nulos del campo release date deben eliminarse.
	RELEASE: ELIMINAR NULOS
	De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_yeardonde extraerán el año de la fecha de estreno.
	CAMPOS FECHA: FORMATEAR A AAAA-MM-DD, CREAR COLUMNA RELEASE_YEAR DE LA FECHA DE ESTRENO
	Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.
	CREAR RETURN: REVENUE/BUDGET, 0 si no hay datos suficientes
	Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.
	ELIMINAR VIDEO,IMDB_ID,ADULT,ORIGINAL_TITLE,POSTER_PATH Y HOMEPAGE.
	
Funciones	
	def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
	Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X
	def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
	Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X
	def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
	Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X
	def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
	Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de Xvaloraciones, con un promedio de X
	def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado cx vy el promedio de retorno. La definición no deberá considerar directores.
	Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación
	def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.
	
Eda	
	Ver si hay outliers o anomalías (que no tienen que ser errores necesariamente 👀 ), y ver si hay algún patrón interesante que valga la pena explorar en un análisis posterior. 
	Las nubes de palabras dan una buena idea de cuáles palabras son más frecuentes en los títulos, ¡podría ayudar al sistema de recomendación!😉
	El EDA debería incluir gráficas interesantes para extraer datos, como por ejemplo una nube de palabras con las palabras más frecuentes en los títulos de las películas.
	
Sistema de recomendación	
	Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score de similaridad y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse:
	def recomendacion( titulo ): Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.


In [1]:
import pandas as pd
import numpy as np
import ast
from pandas import json_normalize
import os


import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# Guarda el valor actual de 'display.max_colwidth'
original_max_colwidth = pd.get_option('display.max_colwidth')

# Establece el nuevo valor para 'display.max_colwidth' para poder visualizar correctamente los valores de las columnas
pd.set_option('display.max_colwidth', None)

In [59]:
#Creamos una funcion para mostrar los tipos de columnas, cantidad de filas nulas y % de nulos
def resumen(df):
    print(f'data shape: {df.shape}')
    resumen = pd.DataFrame(df.dtypes, columns=['data type'])
    resumen['#missing'] = df.isnull().sum()
    resumen['%missing'] = df.isnull().sum()*100/len(df)
    #resumen['#unique'] = df.nunique()
    return resumen

Leo los datos de los archivos Credits y Movies y los almaceno en datasets

In [3]:
df_credits = pd.read_csv('credits.csv')

In [4]:
# crear una columna index
df_credits.reset_index(drop=False, inplace=True)
# Renombrar la columna 'id' 
df_credits = df_credits.rename(columns={'id': 'movie_id'})

print(df_credits.columns)
print(df_credits.shape)
print(df_credits.head)

Index(['index', 'cast', 'crew', 'movie_id'], dtype='object')
<bound method DataFrame.info of        index  \
0          0   
1          1   
2          2   
3          3   
4          4   
...      ...   
45471  45471   
45472  45472   
45473  45473   
45474  45474   
45475  45475   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [62]:
df_movies = pd.read_csv('movies_dataset.csv')
# Renombrar la columna 'id' 
df_movies = df_movies.rename(columns={'id': 'movie_id'})
df_movies['movie_id'] = pd.to_numeric(df_movies['movie_id'], errors='coerce')

print(df_movies.columns)
print(df_movies.shape)
resumen(df_movies)
print(df_movies.head)


/var/folders/83/mc_56f2n2y7bfw5_shj4jwg80000gn/T/ipykernel_807/1591644291.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('movies_dataset.csv')


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'movie_id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
(45466, 24)
data shape: (45466, 24)
<bound method NDFrame.head of        adult  \
0      False   
1      False   
2      False   
3      False   
4      False   
...      ...   
45461  False   
45462  False   
45463  False   
45464  False   
45465  False   

                                                                                                                                                 belongs_to_collection  \
0                {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.j

TRANSFORMACIONES

1. Desanidamos las columnas con listas de los datasets 
Credits: 
    -cast 
    -crew

In [6]:
# Desanidamos crew

# Manejar NaN reemplazándolos con una cadena vacía
df_credits['crew'] = df_credits['crew'].fillna('[]')

# Convertir en lista la columna cast y crew
df_credits['crew'] = df_credits['crew'].apply(ast.literal_eval)

# Expandir las listas en filas
df_exploded_crew = df_credits.explode('crew')

# Verificar el DataFrame después de la expansión de crew
print("\nDataFrame después de la expansión de crew:")
print(df_exploded_crew.columns)
print(df_exploded_crew.shape)
print(df_exploded_crew.head)


DataFrame después de la expansión de crew:
Index(['index', 'cast', 'crew', 'movie_id'], dtype='object')
(465085, 4)
<bound method NDFrame.head of        index  \
0          0   
0          0   
0          0   
0          0   
0          0   
...      ...   
45473  45473   
45473  45473   
45474  45474   
45474  45474   
45475  45475   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [7]:
# Expandir los diccionarios en columnas 
df_normalized_crew = json_normalize(df_exploded_crew['crew'])

# Renombrar algunas columnas
df_normalized_crew = df_normalized_crew.rename(columns={'id': 'crew_id'})
df_normalized_crew = df_normalized_crew.rename(columns={'job': 'crew_job'})
df_normalized_crew = df_normalized_crew.rename(columns={'name': 'crew_name'})

#Verificamos
print(df_normalized_crew.columns)
print(df_normalized_crew.shape)
print(df_normalized_crew.head)

Index(['credit_id', 'department', 'gender', 'crew_id', 'crew_job', 'crew_name',
       'profile_path'],
      dtype='object')
(465085, 7)
<bound method NDFrame.head of                        credit_id  department  gender    crew_id  \
0       52fe4284c3a36847f8024f49   Directing     2.0     7879.0   
1       52fe4284c3a36847f8024f4f     Writing     2.0    12891.0   
2       52fe4284c3a36847f8024f55     Writing     2.0        7.0   
3       52fe4284c3a36847f8024f5b     Writing     2.0    12892.0   
4       52fe4284c3a36847f8024f61     Writing     0.0    12893.0   
...                          ...         ...     ...        ...   
465080  52fe4776c3a368484e0c8399       Sound     0.0   549356.0   
465081  52fe4776c3a368484e0c839f      Camera     2.0    58818.0   
465082  533bccebc3a36844cf0011a7   Directing     0.0  1085341.0   
465083  58ebbc26925141281908aa0a  Production     2.0  1195656.0   
465084  593e676c92514105b702e68e   Directing     0.0  1344495.0   

                       crew

In [8]:
# Analizando los requerimientos de la API, eliminaremos algunas columnas desanidadas que no seran necesarias para lograr el objetivo
df_normalized_crew.drop(columns=['credit_id', 'department', 'gender', 'profile_path'], inplace=True)
print(df_normalized_crew.columns)
print(df_normalized_crew.shape)

Index(['crew_id', 'crew_job', 'crew_name'], dtype='object')
(465085, 3)


In [9]:
df_desanidado_crew = df_exploded_crew.reset_index(drop=True).join(df_normalized_crew)
df_desanidado_crew.drop(columns=['crew','cast'], inplace=True)
print(df_desanidado_crew.columns)
print(df_desanidado_crew.shape)
print(df_desanidado_crew.head)

Index(['index', 'movie_id', 'crew_id', 'crew_job', 'crew_name'], dtype='object')
(465085, 5)
<bound method NDFrame.head of         index  movie_id    crew_id                 crew_job  \
0           0       862     7879.0                 Director   
1           0       862    12891.0               Screenplay   
2           0       862        7.0               Screenplay   
3           0       862    12892.0               Screenplay   
4           0       862    12893.0               Screenplay   
...       ...       ...        ...                      ...   
465080  45473     67758   549356.0  Original Music Composer   
465081  45473     67758    58818.0  Director of Photography   
465082  45474    227506  1085341.0                 Director   
465083  45474    227506  1195656.0                 Producer   
465084  45475    461257  1344495.0                 Director   

                  crew_name  
0             John Lasseter  
1               Joss Whedon  
2            Andrew Stanton  


In [10]:
# Desanidamos cast

# Manejar NaN reemplazándolos con una cadena vacía
df_credits['cast'] = df_credits['cast'].fillna('[]')

# Convertir en lista la columna cast
df_credits['cast'] = df_credits['cast'].apply(ast.literal_eval)

# Expandir las listas en filas
df_exploded_cast = df_credits.explode('cast')

# Verificar el DataFrame después de la expansión de cast
print("\nDataFrame después de la expansión de cast:")
print(df_exploded_cast.columns)
print(df_exploded_cast.shape)



DataFrame después de la expansión de cast:
Index(['index', 'cast', 'crew', 'movie_id'], dtype='object')
(564892, 4)


In [11]:
# Expandir los diccionarios en columnas 
df_normalized_cast = json_normalize(df_exploded_cast['cast'])

# Renombrar algunas columnas
df_normalized_cast = df_normalized_cast.rename(columns={'id': 'actor_id'})
df_normalized_cast = df_normalized_cast.rename(columns={'name': 'actor_name'})

# Verificamos
print(df_normalized_cast.columns)
print(df_normalized_cast.shape)
print(df_normalized_cast.head)

Index(['cast_id', 'character', 'credit_id', 'gender', 'actor_id', 'actor_name',
       'order', 'profile_path'],
      dtype='object')
(564892, 8)
<bound method NDFrame.head of         cast_id                character                 credit_id  gender  \
0          14.0            Woody (voice)  52fe4284c3a36847f8024f95     2.0   
1          15.0   Buzz Lightyear (voice)  52fe4284c3a36847f8024f99     2.0   
2          16.0  Mr. Potato Head (voice)  52fe4284c3a36847f8024f9d     2.0   
3          17.0       Slinky Dog (voice)  52fe4284c3a36847f8024fa1     2.0   
4          18.0              Rex (voice)  52fe4284c3a36847f8024fa5     2.0   
...         ...                      ...                       ...     ...   
564887      3.0                           52fe4ea59251416c7515d7d9     1.0   
564888      4.0                           52fe4ea59251416c7515d7dd     2.0   
564889      5.0                           52fe4ea59251416c7515d7e1     0.0   
564890      6.0                           5

In [12]:
# Analizando los requerimientos de la API, eliminaremos algunas columnas desanidadas que no seran necesarias para lograr el objetivo
df_normalized_cast.drop(columns=['cast_id', 'credit_id', 'gender', 'order', 'profile_path'], inplace=True)
print(df_normalized_cast.columns)

Index(['character', 'actor_id', 'actor_name'], dtype='object')


In [13]:
# Combinar la desanidacion de cast 
df_desanidado_cast = df_exploded_cast.reset_index(drop=True).join(df_normalized_cast)
df_desanidado_cast.drop(columns=['cast','crew'], inplace=True)
print(df_desanidado_cast.columns)
print(df_desanidado_cast.shape)
print(df_desanidado_cast.head)

Index(['index', 'movie_id', 'character', 'actor_id', 'actor_name'], dtype='object')
(564892, 5)
<bound method NDFrame.head of         index  movie_id                character   actor_id         actor_name
0           0       862            Woody (voice)       31.0          Tom Hanks
1           0       862   Buzz Lightyear (voice)    12898.0          Tim Allen
2           0       862  Mr. Potato Head (voice)     7167.0        Don Rickles
3           0       862       Slinky Dog (voice)    12899.0         Jim Varney
4           0       862              Rex (voice)    12900.0      Wallace Shawn
...       ...       ...                      ...        ...                ...
564887  45474    227506                           1090923.0  Nathalie Lissenko
564888  45474    227506                           1136422.0       Pavel Pavlov
564889  45474    227506                           1261758.0  Aleksandr Chabrov
564890  45474    227506                             29199.0        Vera Orlova
56489

In [14]:
# Unir los dos DataFrames
df_credits_final = pd.concat([df_desanidado_cast, df_desanidado_crew])
print(df_credits_final.columns)
print(df_credits_final.shape)
print(df_credits_final.head)

Index(['index', 'movie_id', 'character', 'actor_id', 'actor_name', 'crew_id',
       'crew_job', 'crew_name'],
      dtype='object')
(1029977, 8)
<bound method NDFrame.head of         index  movie_id                character  actor_id     actor_name  \
0           0       862            Woody (voice)      31.0      Tom Hanks   
1           0       862   Buzz Lightyear (voice)   12898.0      Tim Allen   
2           0       862  Mr. Potato Head (voice)    7167.0    Don Rickles   
3           0       862       Slinky Dog (voice)   12899.0     Jim Varney   
4           0       862              Rex (voice)   12900.0  Wallace Shawn   
...       ...       ...                      ...       ...            ...   
465080  45473     67758                      NaN       NaN            NaN   
465081  45473     67758                      NaN       NaN            NaN   
465082  45474    227506                      NaN       NaN            NaN   
465083  45474    227506                      NaN      

In [15]:
df_credits_final = df_credits_final.drop_duplicates()
print(df_credits_final.columns)
print(df_credits_final.shape)
print(df_credits_final.head)

Index(['index', 'movie_id', 'character', 'actor_id', 'actor_name', 'crew_id',
       'crew_job', 'crew_name'],
      dtype='object')
(1029552, 8)
<bound method NDFrame.head of         index  movie_id                character  actor_id     actor_name  \
0           0       862            Woody (voice)      31.0      Tom Hanks   
1           0       862   Buzz Lightyear (voice)   12898.0      Tim Allen   
2           0       862  Mr. Potato Head (voice)    7167.0    Don Rickles   
3           0       862       Slinky Dog (voice)   12899.0     Jim Varney   
4           0       862              Rex (voice)   12900.0  Wallace Shawn   
...       ...       ...                      ...       ...            ...   
465080  45473     67758                      NaN       NaN            NaN   
465081  45473     67758                      NaN       NaN            NaN   
465082  45474    227506                      NaN       NaN            NaN   
465083  45474    227506                      NaN      

Ahora trabajamos con las variables de listas de Movies: 
    -belongs_to_collection
    -genres,
    -spoken_languages
    -production_countries
    -production_companies

In [16]:
# Desanidamos belongs to collection

# Manejar NaN reemplazándolos con una cadena vacía
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].fillna("[]")

# Asegúrate de que las entradas vacías sean listas válidas
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(lambda x: [] if x == "[]" else ast.literal_eval(x))

print("\nDataFrame después de manejar entradas vacías:")
print(df_movies)

# Solo toma las filas donde 'belongs_to_collection' no está vacío
non_empty_collections = df_movies[df_movies['belongs_to_collection'].apply(lambda x: isinstance(x, dict))]

print("\nDataFrame después de manejar entradas vacías:")
print(non_empty_collections.columns)
print(non_empty_collections.shape)

# Aplicar json_normalize solo a las filas no vacías
df_normalized_coll = json_normalize(non_empty_collections['belongs_to_collection'])
df_normalized_coll = df_normalized_coll.rename(columns={'name': 'belongs_to_collection'})

# Analizando los requerimientos de la API, eliminaremos algunas columnas desanidadas que no seran necesarias para lograr el objetivo
df_normalized_coll.drop(columns=['poster_path', 'backdrop_path', 'id'], inplace=True)


print("\nDataFrame después de normalizar 'belongs_to_collection':")
print(df_normalized_coll)

# Combinar el DataFrame normalizado con el original
df_non_empty = non_empty_collections.reset_index(drop=True)
df_desanidado_coll = df_non_empty.drop(columns=['belongs_to_collection']).reset_index(drop=True).join(df_normalized_coll)

print("\nDataFrame colecction desanidado:")
print(df_desanidado_coll.columns)
print(df_desanidado_coll.shape)
print(df_desanidado_coll.head)

# Crear un DataFrame para las filas con belongs_to_collection vacío
empty_collections = df_movies[df_movies['belongs_to_collection'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

# Concatenar los DataFrames manteniendo el orden original
df_combined_1 = pd.concat([df_desanidado_coll, empty_collections], ignore_index=True).sort_index()
print("\nDataFrame combinado final p1:")
print(df_combined_1.columns)
print(df_combined_1.shape)
print(df_combined_1.head)


DataFrame después de manejar entradas vacías:
       adult  \
0      False   
1      False   
2      False   
3      False   
4      False   
...      ...   
45461  False   
45462  False   
45463  False   
45464  False   
45465  False   

                                                                                                                                                 belongs_to_collection  \
0                {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}   
1                                                                                                                                                                   []   
2          {'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet.jpg', 'backdrop_path': '/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg'}   
3                                                                               

In [17]:
# Desanidamos production_companies

# Manejar NaN reemplazándolos con una cadena vacía
df_combined_1['production_companies'] = df_combined_1['production_companies'].fillna('[]')

# Convertir en lista la columna production_companies
df_combined_1['production_companies'] = df_combined_1['production_companies'].apply(ast.literal_eval)

# Expandir las listas en filas
df_exploded_prodco = df_combined_1.explode('production_companies')

# Verificar el DataFrame después de la expansión de production_companies
print("\nDataFrame después de la expansión de production_companies:")
print(df_exploded_prodco.columns)
print(df_exploded_prodco.shape)
print(df_exploded_prodco.head)


DataFrame después de la expansión de production_companies:
Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'id_collection', 'belongs_to_collection'],
      dtype='object')
(82423, 25)
<bound method NDFrame.head of        adult    budget  \
0      False  30000000   
1      False         0   
1      False         0   
2      False         0   
2      False         0   
...      ...       ...   
45458  False         0   
45459  False         0   
45460  False         0   
45461  False         0   
45462  False         0   

                                                                                                 genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name'

In [18]:
# Expandir los diccionarios en columnas 
df_normalized_prodco = json_normalize(df_exploded_prodco['production_companies'])

# Renombrar algunas columnas
df_normalized_prodco = df_normalized_prodco.rename(columns={'name': 'production_company'})

# Analizando los requerimientos de la API, eliminaremos algunas columnas desanidadas que no seran necesarias para lograr el objetivo
df_normalized_prodco.drop(columns=['id'], inplace=True)

# Verificamos
print(df_normalized_prodco.columns)
print(df_normalized_prodco.shape)
print(df_normalized_prodco.head)

Index(['production_company'], dtype='object')
(82423, 1)
<bound method NDFrame.head of             production_company
0      Pixar Animation Studios
1                 Warner Bros.
2               Lancaster Gate
3        Sandollar Productions
4          Touchstone Pictures
...                        ...
82418                      NaN
82419              Sine Olivia
82420  American World Pictures
82421                Yermoliev
82422                      NaN

[82423 rows x 1 columns]>


In [19]:
# Combinar la desanidacion de production_companies 
df_combined_2= df_exploded_prodco.reset_index(drop=True).join(df_normalized_prodco)
df_combined_2.drop(columns=['production_companies'], inplace=True)
print(df_combined_2.columns)
print(df_combined_2.shape)
print(df_combined_2.head)

Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'id_collection', 'belongs_to_collection',
       'production_company'],
      dtype='object')
(82423, 25)
<bound method NDFrame.head of        adult    budget  \
0      False  30000000   
1      False         0   
2      False         0   
3      False         0   
4      False         0   
...      ...       ...   
82418  False         0   
82419  False         0   
82420  False         0   
82421  False         0   
82422  False         0   

                                                                                                 genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]   
1            

In [20]:
# Desanidamos production_companies

# Manejar NaN reemplazándolos con una cadena vacía
df_combined_2['production_countries'] = df_combined_2['production_countries'].fillna('[]')

# Convertir en lista la columna production_countries
df_combined_2['production_countries'] = df_combined_2['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expandir las listas en filas
df_exploded_prodcty = df_combined_2.explode('production_countries')

# Verificar el DataFrame después de la expansión de production_countries
print("\nDataFrame después de la expansión de production_countries:")
print(df_exploded_prodcty.columns)
print(df_exploded_prodcty.shape)
print(df_exploded_prodcty.head)

# Expandir los diccionarios en columnas 
df_normalized_prodcty = json_normalize(df_exploded_prodcty['production_countries'])

# Renombrar algunas columnas
df_normalized_prodcty = df_normalized_prodcty.rename(columns={'name': 'production_country'})


DataFrame después de la expansión de production_countries:
Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'id_collection', 'belongs_to_collection',
       'production_company'],
      dtype='object')
(118445, 25)
<bound method NDFrame.head of        adult    budget  \
0      False  30000000   
1      False         0   
2      False         0   
3      False         0   
4      False         0   
...      ...       ...   
82418  False         0   
82419  False         0   
82420  False         0   
82421  False         0   
82422  False         0   

                                                                                                 genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name':

In [21]:
# Analizando los requerimientos de la API, eliminaremos algunas columnas desanidadas que no seran necesarias para lograr el objetivo
df_normalized_prodcty.drop(columns=['iso_3166_1'], inplace=True)

# Verificamos
print(df_normalized_prodcty.columns)
print(df_normalized_prodcty.shape)
print(df_normalized_prodcty.head)


# Combinar la desanidacion de production_companies 
df_combined_3= df_exploded_prodcty.reset_index(drop=True).join(df_normalized_prodcty)
df_combined_3.drop(columns=['production_countries'], inplace=True)
print(df_combined_3.columns)
print(df_combined_3.shape)
print(df_combined_3.head)

Index(['production_country'], dtype='object')
(118445, 1)
<bound method NDFrame.head of               production_country
0       United States of America
1       United States of America
2       United States of America
3       United States of America
4       United States of America
...                          ...
118440                      Iran
118441               Philippines
118442  United States of America
118443                    Russia
118444            United Kingdom

[118445 rows x 1 columns]>
Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'release_date', 'revenue', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'id_collection', 'belongs_to_collection', 'production_company',
       'production_country'],
      dtype='object')
(118445, 25)
<bound method NDFrame.head of         adult    budget  \
0 

In [22]:
# Desanidamos spoken_languages

# Manejar NaN reemplazándolos con una cadena vacía
df_combined_3['spoken_languages'] = df_combined_3['spoken_languages'].fillna('[]')

# Convertir en lista la columna spoken_languages
df_combined_3['spoken_languages'] = df_combined_3['spoken_languages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expandir las listas en filas
df_exploded_spkl = df_combined_3.explode('spoken_languages')

# Verificar el DataFrame después de la expansión de spoken_languages
print("\nDataFrame después de la expansión de spoken_languages:")
print(df_exploded_spkl.columns)
print(df_exploded_spkl.shape)
print(df_exploded_spkl.head)

# Expandir los diccionarios en columnas 
df_normalized_spkl = json_normalize(df_exploded_spkl['spoken_languages'])

# Renombrar la columna
df_normalized_spkl = df_normalized_spkl.rename(columns={'name': 'spoken_language'})


DataFrame después de la expansión de spoken_languages:
Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'release_date', 'revenue', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'id_collection', 'belongs_to_collection', 'production_company',
       'production_country'],
      dtype='object')
(186576, 25)
<bound method NDFrame.head of         adult    budget  \
0       False  30000000   
1       False         0   
2       False         0   
3       False         0   
4       False         0   
...       ...       ...   
118440  False         0   
118441  False         0   
118442  False         0   
118443  False         0   
118444  False         0   

                                                                                                  genres  \
0       [{'id': 16, 'name': 'Animation'}, {'id': 35,

In [23]:
# Analizando los requerimientos de la API, eliminaremos algunas columnas desanidadas que no seran necesarias para lograr el objetivo
df_normalized_spkl.drop(columns=['iso_639_1'], inplace=True)

# Verificamos
print(df_normalized_spkl.columns)
print(df_normalized_spkl.shape)
print(df_normalized_spkl.head)


# Combinar la desanidacion de spoken_languages 
df_combined_4= df_exploded_spkl.reset_index(drop=True).join(df_normalized_spkl)
df_combined_4.drop(columns=['spoken_languages'], inplace=True)
print(df_combined_4.columns)
print(df_combined_4.shape)
print(df_combined_4.head)

Index(['spoken_language'], dtype='object')
(186576, 1)
<bound method NDFrame.head of        spoken_language
0              English
1              English
2              English
3              English
4              English
...                ...
186571           فارسی
186572                
186573         English
186574             NaN
186575         English

[186576 rows x 1 columns]>
Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'id_collection', 'belongs_to_collection', 'production_company',
       'production_country', 'spoken_language'],
      dtype='object')
(186576, 25)
<bound method NDFrame.head of         adult    budget  \
0       False  30000000   
1       False         0   
2       False         0   
3       False         0   
4       False     

In [24]:
# Por ultimo desanidamos genres

# Manejar NaN reemplazándolos con una cadena vacía
df_combined_4['genres'] = df_combined_4['genres'].fillna('[]')

# Convertir en lista la columna spoken_languages
df_combined_4['genres'] = df_combined_4['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expandir las listas en filas
df_exploded_genres = df_combined_4.explode('genres')

# Verificar el DataFrame después de la expansión de genres
print("\nDataFrame después de la expansión de genres:")
print(df_exploded_genres.columns)
print(df_exploded_genres.shape)
print(df_exploded_genres.head)

# Expandir los diccionarios en columnas 
df_normalized_genres = json_normalize(df_exploded_genres['genres'])

# Renombrar la columna
df_normalized_genres = df_normalized_genres.rename(columns={'name': 'genre'})


DataFrame después de la expansión de genres:
Index(['adult', 'budget', 'genres', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'id_collection', 'belongs_to_collection', 'production_company',
       'production_country', 'spoken_language'],
      dtype='object')
(419767, 25)
<bound method NDFrame.head of         adult    budget                            genres  \
0       False  30000000   {'id': 16, 'name': 'Animation'}   
0       False  30000000      {'id': 35, 'name': 'Comedy'}   
0       False  30000000   {'id': 10751, 'name': 'Family'}   
1       False         0  {'id': 10749, 'name': 'Romance'}   
1       False         0      {'id': 35, 'name': 'Comedy'}   
...       ...       ...                               ...   
186573  False         0      {'id': 28, 'name': 'Action'}   
186573  

In [25]:
# Analizando los requerimientos de la API, eliminaremos algunas columnas desanidadas que no seran necesarias para lograr el objetivo
df_normalized_genres.drop(columns=['id'], inplace=True)

# Verificamos
print(df_normalized_genres.columns)
print(df_normalized_genres.shape)
print(df_normalized_genres.head)


# Combinar la desanidacion de production_companies 
df_movies_final= df_exploded_genres.reset_index(drop=True).join(df_normalized_genres)
df_movies_final.drop(columns=['genres'], inplace=True)
print(df_movies_final.columns)
print(df_movies_final.shape)
print(df_movies_final.head)

Index(['genre'], dtype='object')
(419767, 1)
<bound method NDFrame.head of             genre
0       Animation
1          Comedy
2          Family
3         Romance
4          Comedy
...           ...
419762     Action
419763      Drama
419764   Thriller
419765        NaN
419766        NaN

[419767 rows x 1 columns]>
Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'id_collection',
       'belongs_to_collection', 'production_company', 'production_country',
       'spoken_language', 'genre'],
      dtype='object')
(419767, 25)
<bound method NDFrame.head of         adult    budget                              homepage      id  \
0       False  30000000  http://toystory.disney.com/toy-story     862   
1       False  30000000  http://toystory.disney.com/toy-story     862   
2    

2. RELLENAR NULOS. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [42]:
#Verificamos las caracteristicas de los datos
resumen(df_movies_final)

data shape: (419588, 25)


,data type,#missing,%missing
adult,object,0,0.000000
budget,int64,0,0.000000
homepage,object,291067,69.369715
id,object,0,0.000000
imdb_id,object,61,0.014538
original_language,object,96,0.022880
original_title,object,0,0.000000
overview,object,2495,0.594631
popularity,object,0,0.000000
poster_path,object,519,0.123693


In [41]:
#Forzamos que revenue y budget sean columnas numericas
df_movies_final['revenue'] = pd.to_numeric(df_movies_final['revenue'], errors='coerce')
df_movies_final['budget'] = pd.to_numeric(df_movies_final['budget'], errors='coerce')

In [44]:
df_movies_final['revenue'] = df_movies_final['revenue'].fillna(0)
df_movies_final['budget'] = df_movies_final['budget'].fillna(0)

3. RELEASE: ELIMINAR NULOS. Los valores nulos del campo release date deben eliminarse.

In [29]:
df_movies_final = df_movies_final.dropna(subset=['release_date'])

4. CAMPOS FECHA: FORMATEAR A AAAA-MM-DD, CREAR COLUMNA RELEASE_YEAR DE LA FECHA DE ESTRENO. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_yeardonde extraerán el año de la fecha de estreno.

In [32]:
print(df_movies_final['release_date'].head)

<bound method NDFrame.head of 0         1995-10-30
1         1995-10-30
2         1995-10-30
3         1995-12-22
4         1995-12-22
             ...    
419762    2003-08-01
419763    2003-08-01
419764    2003-08-01
419765    1917-10-21
419766    2017-06-09
Name: release_date, Length: 419588, dtype: object>


In [30]:
print(type(df_movies_final["release_date"]))

<class 'pandas.core.series.Series'>


In [33]:
# Convertir la columna al tipo datetime
df_movies_final['release_date'] = pd.to_datetime(df_movies_final['release_date'])

# Asegurarse de que las fechas están en el formato 'AAAA-MM-DD'
df_movies_final['release_date'] = df_movies_final['release_date'].dt.strftime('%Y-%m-%d')

In [34]:
print(df_movies_final['release_date'].head)

<bound method NDFrame.head of 0         1995-10-30
1         1995-10-30
2         1995-10-30
3         1995-12-22
4         1995-12-22
             ...    
419762    2003-08-01
419763    2003-08-01
419764    2003-08-01
419765    1917-10-21
419766    2017-06-09
Name: release_date, Length: 419588, dtype: object>


CREAR RETURN: REVENUE/BUDGET. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [46]:
# Calcular el retorno de inversión (revenue / budget) y asignar 0 cuando no hay datos
df_movies_final['return'] = df_movies_final.apply(lambda row: row['revenue'] / row['budget'] if pd.notnull(row['revenue']) and pd.notnull(row['budget']) and row['budget'] != 0 else 0, axis=1)

In [49]:
print(df_movies_final[df_movies_final['return'] != 0][['revenue', 'budget', 'return']].head)

<bound method NDFrame.head of             revenue    budget     return
0       373554033.0  30000000  12.451801
1       373554033.0  30000000  12.451801
2       373554033.0  30000000  12.451801
9       352194034.0  58000000   6.072311
10      352194034.0  58000000   6.072311
...             ...       ...        ...
419497    1268793.0   2000000   0.634397
419564    1413000.0   5000000   0.282600
419565    1413000.0   5000000   0.282600
419566    1413000.0   5000000   0.282600
419567    1413000.0   5000000   0.282600

[118161 rows x 3 columns]>


ELIMINAR ALGUNAS COLUMNAS. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [70]:
#df_movies_final.drop(columns=['adult', 'homepage','original_title', 'poster_path','imdb_id', 'video'], inplace=True)
df_movies_final.drop(columns=['belongs_to_collection'], inplace=True)


In [67]:
# Transformamos el tipo de datos de algunas columnas a lo que representan
df_movies_final['popularity'] = pd.to_numeric(df_movies_final['popularity'], errors='coerce')

In [71]:
resumen(df_movies_final)

data shape: (419588, 18)


,data type,#missing,%missing
budget,int64,0,0.000000
movie_id,int64,0,0.000000
original_language,object,96,0.022880
overview,object,2495,0.594631
popularity,float64,0,0.000000
release_date,object,0,0.000000
revenue,float64,0,0.000000
runtime,float64,544,0.129651
status,object,143,0.034081
tagline,object,192184,45.803026


In [58]:
resumen(df_credits_final)

data shape: (1029552, 8)


,data type,#missing,%missing
index,int64,0,0.000000
movie_id,int64,0,0.000000
character,object,467082,45.367500
actor_id,float64,467082,45.367500
actor_name,object,467082,45.367500
crew_id,float64,565238,54.901355
crew_job,object,565238,54.901355
crew_name,object,565238,54.901355


In [72]:
#Exportamos los datos desanidados en formato parquet

df_movies_pya = pa.Table.from_pandas(df_movies_final)
pq.write_table(df_movies_pya, "df_movies_desanidado.parquet")

df_credits_pya = pa.Table.from_pandas(df_credits_final)
pq.write_table(df_credits_pya, "df_credits_desanidado.parquet")